In [213]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.cross_validation import train_test_split
from sklearn.base import clone
import timeit
import math

In [6]:
def pca(F, X):
    n, d = X.shape
    mu = np.zeros((d, 1))
    Z = np.zeros((d, F))
    for i in range(d):
        mu[i] = (1. / n) * np.sum(X[:, [i]])
    X = X - mu.T
    U, s, Vt = la.svd(X, False)
    g = s[:F]
    for i in range(F):
        g[i] = 1. / g[i]
    W = Vt[:F]
    Z = np.dot(W.T, np.diag(g))
    return (mu, Z)

def pca_proj(X,mu,Z):
    n, d = X.shape
    X = X - mu.T
    return np.dot(X, Z)

In [191]:
def k_fold(k, model, X, y):
    n, d = X.shape
    z = np.zeros((k, 1))
    for i in range(k):
        T = list(range(int((i * n) / k), int((n * (i + 1) / k))))
        S = [j for j in range(n) if j not in T]
        curr_model = clone(model)
        curr_model.fit(X[S], y[S])
        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(X[T])) ** 2)
    return z

In [192]:
def bootstrapping(B, model, X, y):
    n, d = X.shape
    z = np.zeros((B, 1))
    for i in range(B):
        u = np.random.choice(n, n, replace=True)
        S = np.unique(u)
        T = np.setdiff1d(np.arange(n), S, assume_unique=True)
        curr_model = clone(model)
        curr_model.fit(X[u], y[u])
        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        # theta_hat will be d by 1
        z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(X[T])) ** 2)
    return z

In [205]:
def evaluateModel(model, X, y, k=5, B=5):
    ########################KFOLD###################
    print('Evaluating K-fold with %d folds.' % k)
    start_time = timeit.default_timer()
    k_fold_z = k_fold(k, model, X, y)
    elapsed = timeit.default_timer() - start_time
    
    k_fold_mse = np.mean(k_fold_z)
    print('K-fold Mean Squared Error: ', k_fold_mse)
    
    k_fold_rmse = math.sqrt(k_fold_mse)
    print('K-fold Square Root Mean Squared Error: ', k_fold_rmse)

    print("Time elapsed for k-fold: ", elapsed)
    
    print()
    print()
    ###################BOOTSTRAPPING################
    print('Evaluating bootstrapping with %d bootstraps.' % B)
    start_time = timeit.default_timer()
    bootstrapping_z = bootstrapping(B, model, X, y)
    elapsed = timeit.default_timer() - start_time
    
    bootstrapping_mse = np.mean(bootstrapping_z)
    print('Bootstrapping Mean Squared Error: ', bootstrapping_mse)
    
    bootstrapping_rmse = math.sqrt(bootstrapping_mse)
    print('Bootstrapping Square Root Mean Squared Error: ', bootstrapping_rmse)

    print("Time elapsed for bootstrapping: ", elapsed)
    
    return (k_fold_z, bootstrapping_z)

# Data Processing

In [34]:
data = pd.read_csv("train.csv", header=0)
print(data.shape)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]

print(X.shape)
print(Y.shape)

(1460, 81)
(1460, 80)
(1460, 1)


In [38]:
# this just sums up how many nulls per feature and divides to find percentage of nulls per feature
# if over 50% null then print the feature
data_keys = X.keys()
for i, b in enumerate((X.isnull().sum() / X.shape[0]) > 0.5):
    if b:
        print(data_keys[i])

Alley
PoolQC
Fence
MiscFeature


In [8]:
# data = data.drop(['Alley', 'MiscFeature', 'Fence', 'PoolQC'], axis=1)

In [7]:
# Replaces categorical value in Quality columns with numerical scale
qualityCols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
              'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']

data[qualityCols].head()

for col in qualityCols:
    # NA is never used since all NA's got converted to NaN objects when pandas read in the csv
    data[col] = data[col].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA': 0})

data[qualityCols].head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond
0,4,3,4.0,3.0,5,4,NaN,3.0,3.0
1,3,3,4.0,3.0,5,3,3.0,3.0,3.0
2,4,3,4.0,3.0,5,4,3.0,3.0,3.0
3,3,3,3.0,4.0,4,4,4.0,3.0,3.0
4,4,3,4.0,3.0,5,4,3.0,3.0,3.0


In [39]:
# categorical columns
catCols = set(list(X))-set(list(X._get_numeric_data()))
print(catCols)

# #TRY dropping all cat cols
# data = data.drop(columns=catCols)

{'FireplaceQu', 'LotShape', 'Fence', 'GarageCond', 'PoolQC', 'Exterior1st', 'LandContour', 'Foundation', 'Condition1', 'ExterQual', 'Electrical', 'Street', 'LotConfig', 'BsmtQual', 'GarageQual', 'PavedDrive', 'BsmtExposure', 'Condition2', 'BldgType', 'MasVnrType', 'SaleCondition', 'RoofStyle', 'RoofMatl', 'Exterior2nd', 'BsmtCond', 'BsmtFinType2', 'GarageType', 'Neighborhood', 'KitchenQual', 'MSZoning', 'Functional', 'CentralAir', 'ExterCond', 'HeatingQC', 'Alley', 'GarageFinish', 'Utilities', 'MiscFeature', 'LandSlope', 'Heating', 'SaleType', 'BsmtFinType1', 'HouseStyle'}


In [40]:
#Perform one hot encoding on all categorical columns
frames = []
for col in catCols:
    oneHot_encoded = pd.get_dummies(X[col])
    oneHot_encoded = oneHot_encoded.add_prefix(col + '_is_')
    frames.append(oneHot_encoded)

X = X.drop(catCols, axis=1)

X = pd.concat(frames, axis=1)

In [41]:
X.keys()

Index(['FireplaceQu_is_Ex', 'FireplaceQu_is_Fa', 'FireplaceQu_is_Gd',
       'FireplaceQu_is_Po', 'FireplaceQu_is_TA', 'LotShape_is_IR1',
       'LotShape_is_IR2', 'LotShape_is_IR3', 'LotShape_is_Reg',
       'Fence_is_GdPrv',
       ...
       'BsmtFinType1_is_Rec', 'BsmtFinType1_is_Unf', 'HouseStyle_is_1.5Fin',
       'HouseStyle_is_1.5Unf', 'HouseStyle_is_1Story', 'HouseStyle_is_2.5Fin',
       'HouseStyle_is_2.5Unf', 'HouseStyle_is_2Story', 'HouseStyle_is_SFoyer',
       'HouseStyle_is_SLvl'],
      dtype='object', length=252)

In [47]:
X.isnull().values.any()

False

In [48]:
# 80:20 train test ratio
test_size = 0.2
# This function splits the training and target sets into random train and test subsets.
# X_train and X_test are subsets of the training data
# y_train and y_test are subsets the the target data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

# PCA Feature Selection

In [164]:
F = 30

In [81]:
X_mu, X_Z = pca(F, X.values)
X_pca = pca_proj(X.values, X_mu, X_Z)

In [83]:
print(X_mu.shape)
print(X_Z.shape)
print(X_pca.shape)

(252, 1)
(252, 30)
(1460, 30)


In [50]:
X_train_mu, X_train_Z = pca(F, X_train.values)

In [55]:
print(X_train_mu.shape)
print(X_train_Z.shape)

(252, 1)
(252, 30)


In [72]:
X_train_pca = pca_proj(X_train.values, X_train_mu, X_train_Z)
print(X_train_pca.shape)

X_test_pca = pca_proj(X_test.values, X_train_mu, X_train_Z)
print(X_test_pca.shape)

(1168, 30)
(292, 30)


# AdaBoost

In [206]:
from sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor()
evaluateModel(adaBoost, X_pca, Y.values.ravel(), k=5, B=5)

adaBoost.fit(X_train_pca, y_train.values.ravel())
adaBoost.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  3024650675.54
K-fold Square Root Mean Squared Error:  54996.82423141105
Time elapsed for k-fold:  1.7181399000000965


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  3044386180.63
Bootstrapping Square Root Mean Squared Error:  55175.95654476395
Time elapsed for bootstrapping:  1.74020819999987


0.48174652703429927

In [112]:
#View Predicted values
predicted = adaBoost.predict(X_test_pca)
ada_pred = y_test.copy()
ada_pred['predicted'] = predicted
ada_pred.head()

,SalePrice,predicted
466,167000,148436.905724
758,179400,196773.623604
350,318061,343546.483871
774,395000,299211.613065
163,103200,136176.105769


# XGBoost Regressor

In [19]:
#!pip3 install xgboost

In [210]:
from xgboost import XGBRegressor

xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)

evaluateModel(xgb, X_pca, Y.values.ravel(), k=5, B=5)

xgb.fit(X_train_pca, y_train)
xgb.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  2110356773.55
K-fold Square Root Mean Squared Error:  45938.61963049125
Time elapsed for k-fold:  1.1101891999996951


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  2449252376.33
Bootstrapping Square Root Mean Squared Error:  49489.92196734414
Time elapsed for bootstrapping:  0.9726442000001043


0.65019408903127041

In [110]:
predicted = xgb.predict(X_test_pca)
xgb_pred = y_test.copy()
xgb_pred['predicted'] = predicted
xgb_pred.head()

,SalePrice,predicted
466,167000,144864.328125
758,179400,160262.921875
350,318061,352198.437500
774,395000,293798.656250
163,103200,112850.273438


# SVM (SVC just to test)

In [176]:
from sklearn import svm

svc_model = svm.SVC(kernel="rbf", C=1.0)

evaluateModel(svc_model, X_pca, Y.values.ravel(), k=5, B=5)

# change C (error) in hypertuning
svc_model.fit(X_train_pca, y_train.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  8060434831.05
K-fold Square Root Mean Squared Error:  89779.92443221048
Time elapsed for k-fold:  10.259636200000386


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  7863722853.71
Bootstrapping Square Root Mean Squared Error:  88677.63446161311
Time elapsed for bootstrapping:  11.805114100000537


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
svc_predicted = svc_model.predict(X_test)
svc_pred = y_test.copy()
svc_pred["predicted"] = svc_predicted
svc_pred.head()

In [124]:
svc_model.score(X_test_pca, y_test.values.ravel())

0.0

# SVM (SVR)

In [212]:
from sklearn import svm

svr_model = svm.SVR(kernel="poly", shrinking=False, coef0=-3500)
# coef0 only works with poly and sigmoid kernels
# it just puts that value instead of the column of 1's

# without it, this model breaks for some reason

evaluateModel(svr_model, X_pca, Y.values.ravel(), k=5, B=5)

# epsilon, degree
svr_model.fit(X_train_pca, y_train.values.ravel())
svr_model.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared Error:  2271964893.39
K-fold Square Root Mean Squared Error:  47665.13288966446
Time elapsed for k-fold:  0.6479687000000922


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  2769887548.46
Bootstrapping Square Root Mean Squared Error:  52629.72115125175
Time elapsed for bootstrapping:  0.8933160000005955


0.63795692564594431

In [119]:
svr_predicted = svr_model.predict(X_test_pca)
svr_pred = y_test.copy()
svr_pred["predicted"] = svr_predicted
svr_pred.head()

,SalePrice,predicted
466,167000,158503.256724
758,179400,137609.404098
350,318061,269615.868976
774,395000,233855.156738
163,103200,124114.390946
